# Build a Simple LLM Application with LCEL
Build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

High level overview of:
- Using language models
- Using PromptTemplates and OutputParsers
- Using LangChain Expression Language (LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying application with LangServe

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10d305950>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d22eb10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal and polite way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a common way to say it to friends or family.)\n* **Coucou, ça va ?** (This is a very informal way to say it, similar to "Hey, what\'s up?")\n\n**Other options:**\n\n* **Bonjour, comment allez-vous aujourd\'hui ?** (Hello, how are you today?)\n* **Ça va ?** (This is a very short and informal way to say "How are you?")\n\n\nLet me know if you\'d like more examples or have any other phrases you want translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 177, 'prompt_tokens': 21, 'total_tokens': 198, 'completion_time': 0.321818182, 'prompt_time': 0.00017219, 'queue_time': 0.014290107000000002, 'total_time': 0.321990372}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal and polite way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a common way to say it to friends or family.)\n* **Coucou, ça va ?** (This is a very informal way to say it, similar to "Hey, what\'s up?")\n\n**Other options:**\n\n* **Bonjour, comment allez-vous aujourd\'hui ?** (Hello, how are you today?)\n* **Ça va ?** (This is a very short and informal way to say "How are you?")\n\n\nLet me know if you\'d like more examples or have any other phrases you want translated!\n'

In [5]:
# Using LCEL - Chain the components
chain = model | parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you\'d like to know how to say other greetings! 😊\n'

In [9]:
# Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"
prompt = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])

In [10]:
result = prompt.invoke({"language": "French", "text": "Hello"})
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [11]:
# Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({"language": "French", "text": "Hello"})

"Hello in French is **Bonjour**. \n\n\nLet me know if you'd like to translate anything else! \n"